# OpenAI API - Moderation (콘텐츠 모니터링)

이 노트북에서는 OpenAI의 **Moderation API**를 사용하여 유해한 콘텐츠(혐오, 폭력, 자해 등)를 탐지하고 필터링하는 방법을 다룹니다.

**학습 목표:**
1. **Moderation 개념 이해:** AI 서비스에서 유해 콘텐츠 필터링이 왜 중요한지 이해하기
2. **유해 카테고리 분류:** 폭력, 혐오, 자해 등 OpenAI가 정의한 유해 카테고리 확인하기
3. **API 실습:** 텍스트와 이미지(Omni 모델)에 포함된 유해성을 감지하고 차단하는 코드 구현하기

# Moderation

[openai-moderation-api-evaluation Dataset](https://huggingface.co/datasets/mmathys/openai-moderation-api-evaluation)

https://platform.openai.com/docs/guides/moderation/overview

https://cookbook.openai.com/examples/how_to_use_moderation

> MS의 Tay 챗봇 AI서비스 중단된 케이스를 통해 Moderation의 중요성을 알 수 있다.
>
> 실제 서비스 오픈전에 가장 많은 테스트를 수행하는 부분이 Moderation이다.
>
> [Microsoft의 챗봇 'Tay' 사건과 한국 이루다 사건 설명](https://www.perplexity.ai/search/microsoftyi-caesbos-tay-sageon-V8X4NMLcSd628JexrxGlAw)


OpenAI의 **Moderation 모델**은 콘텐츠가 OpenAI의 사용 정책을 준수하는지 확인하기 위해 설계된 모델이다. 이 모델은 혐오(hate), 자해(self-harm), 성적인 콘텐츠(sexual content), 폭력(violence) 등 다양한 범주의 콘텐츠를 분류할 수 있는 기능을 제공한다. 텍스트와 이미지의 모니터링에 대한 자세한 내용은 OpenAI의 [Moderation 가이드](https://platform.openai.com/docs/guides/moderation)를 참고할 수 있다.



**모델 및 세부 정보**

| **모델**                     | **최대 토큰 수** | **설명**                                                                 |
|------------------------------|----------------|-------------------------------------------------------------------------|
| **omni-moderation-latest**   | 32,768         | 현재 omni-moderation-2024-09-26을 가리키며, 텍스트와 이미지를 모두 분석할 수 있는 최신 모델이다. |
| **omni-moderation-2024-09-26** | 32,768         | 텍스트와 이미지를 분석할 수 있는 멀티모달(multi-modal) 모니터링 모델의 최신 고정 버전이다.   |
| **text-moderation-latest**   | 32,768         | 현재 text-moderation-007을 가리키는 텍스트 전용 최신 모델이다.                       |
| **text-moderation-stable**   | 32,768         | text-moderation-007을 가리키며 안정적인 텍스트 전용 모델이다.                          |
| **text-moderation-007**      | 32,768         | 이전 세대의 텍스트 전용 모니터링 모델이며, 앞으로는 omni-moderation 모델이 기본으로 추천된다. |

- **멀티모달 분석**: omni-moderation 모델은 텍스트와 이미지 모두를 분석할 수 있어 더욱 강력한 콘텐츠 모니터링을 지원한다.
- **정교한 분류**: 혐오, 자해, 폭력 등 민감한 카테고리에 대한 세부적인 분류를 제공한다.
- **최대 토큰 수**: 모든 모델에서 최대 32,768 토큰을 지원하여 대규모 텍스트 데이터 처리 가능하다.



**콘텐츠 분류**

Moderation API에서 감지할 수 있는 콘텐츠 유형, 지원되는 모델, 입력 형식에 대한 설명이다.


| **카테고리**               | **설명**                                                                                                                   | **지원 모델** | **입력 형식**   |
|----------------------------|----------------------------------------------------------------------------------------------------------------------------|---------------|-----------------|
| **harassment**             | 특정 대상을 괴롭히는 언어를 표현하거나 선동하거나 촉진하는 콘텐츠.                                                          | All           | Text only       |
| **harassment/threatening** | 특정 대상을 괴롭히는 내용 중 폭력이나 심각한 위해를 포함하는 콘텐츠.                                                        | All           | Text only       |
| **hate**                   | 인종, 성별, 민족, 종교, 국적, 성적 지향, 장애 상태 또는 계급에 기반해 증오를 표현, 선동, 촉진하는 콘텐츠.                     | All           | Text only       |
| **hate/threatening**       | 특정 그룹(인종, 성별 등)을 대상으로 폭력 또는 심각한 위해를 포함한 증오 콘텐츠.                                              | All           | Text only       |
| **illicit**                | 불법 행위를 조언하거나 지시하는 콘텐츠. 예: "도둑질하는 방법".                                                               | Omni only     | Text only       |
| **illicit/violent**        | 불법 카테고리에 해당하는 내용 중 폭력이나 무기 조달과 관련된 콘텐츠.                                                         | Omni only     | Text only       |
| **self-harm**              | 자해 행위를 장려하거나 묘사하는 콘텐츠(자살, 자해, 섭식 장애 등).                                                           | All           | Text and image  |
| **self-harm/intent**       | 자해 행위를 하거나 할 의도를 표현한 콘텐츠(자살, 자해, 섭식 장애 등).                                                       | All           | Text and image  |
| **self-harm/instructions** | 자해 행위를 장려하거나 방법을 지시하는 콘텐츠(자살, 자해, 섭식 장애 등).                                                    | All           | Text and image  |
| **sexual**                 | 성적 흥분을 유발하거나 성적 활동을 묘사하거나 성적 서비스를 홍보하는 콘텐츠(성교육 및 웰니스 제외).                          | All           | Text and image  |
| **sexual/minors**          | 18세 미만 개인을 포함하는 성적 콘텐츠.                                                                                      | All           | Text only       |
| **violence**               | 죽음, 폭력, 신체적 부상을 묘사하는 콘텐츠.                                                                                  | All           | Text and images |
| **violence/graphic**       | 죽음, 폭력, 신체적 부상을 **상세히 묘사**한 콘텐츠.                                                                         | All           | Text and images |

- **Text only**: 대부분의 카테고리는 텍스트 기반으로 감지됨.
- **Text and image**: 일부 카테고리(자해, 성적 콘텐츠, 폭력 등)는 이미지도 감지 가능.
- **Omni only**: 불법 콘텐츠는 Omni 모델에서만 지원.


In [2]:
from openai import OpenAI
from dotenv import load_dotenv  # .env 파일의 환경변수 로드
import os                       # 환경변수 접근용

load_dotenv()                   # 현재 위치의 .env를 읽어와 환경변수로 등록
api_key = os.getenv("openai_key")  # .env의 openai_key 값을 가져옴

client = OpenAI(api_key=api_key)

In [3]:
response = client.moderations.create(
    model = 'omni-moderation-latest',
    input = '잔인하고 고통스럽게 XX을 죽이는 방법'
)

response

ModerationCreateResponse(id='modr-8781', model='omni-moderation-latest', results=[Moderation(categories=Categories(harassment=False, harassment_threatening=False, hate=False, hate_threatening=False, illicit=True, illicit_violent=True, self_harm=False, self_harm_instructions=False, self_harm_intent=False, sexual=False, sexual_minors=False, violence=True, violence_graphic=False, harassment/threatening=False, hate/threatening=False, illicit/violent=True, self-harm/intent=False, self-harm/instructions=False, self-harm=False, sexual/minors=False, violence/graphic=False), category_applied_input_types=CategoryAppliedInputTypes(harassment=['text'], harassment_threatening=['text'], hate=['text'], hate_threatening=['text'], illicit=['text'], illicit_violent=['text'], self_harm=['text'], self_harm_instructions=['text'], self_harm_intent=['text'], sexual=['text'], sexual_minors=['text'], violence=['text'], violence_graphic=['text'], harassment/threatening=['text'], hate/threatening=['text'], illic

In [ ]:
print(response.results[0].flagged)           # 최종 차단 권장 여부
print(response.results[0].categories)        # 어떤 카테고리에 해당하는지 True/False
print(response.results[0].category_scores)   # 각 카테고리의 확률/점수

True
Categories(harassment=False, harassment_threatening=False, hate=False, hate_threatening=False, illicit=True, illicit_violent=True, self_harm=False, self_harm_instructions=False, self_harm_intent=False, sexual=False, sexual_minors=False, violence=True, violence_graphic=False, harassment/threatening=False, hate/threatening=False, illicit/violent=True, self-harm/intent=False, self-harm/instructions=False, self-harm=False, sexual/minors=False, violence/graphic=False)
CategoryScores(harassment=0.1417295287243172, harassment_threatening=0.2847851527267333, hate=0.013684915113631835, hate_threatening=0.005384427341632149, illicit=0.684719317244762, illicit_violent=0.6701387999579413, self_harm=0.12318101146642525, self_harm_instructions=0.1811974793642004, self_harm_intent=0.16366083900262193, sexual=0.01982751147780615, sexual_minors=0.00025909793644293347, violence=0.601290163476629, violence_graphic=0.049604810039484366, harassment/threatening=0.2847851527267333, hate/threatening=0.00

In [5]:
# 예시 (차단/대체 응답)
r = response.results[0]

if r.flagged:  # 정책 위반 가능성이 높으면
    print('차단: 유해 콘텐츠로 판단되어 요청을 처리 할 수 없습니다!')  # 사용자 안내 후 차단
    pass
else:
    print('통과 : 정상 처리 진행')  # 후속 로직 진행
    pass

차단: 유해 콘텐츠로 판단되어 요청을 처리 할 수 없습니다!


In [ ]:
import pandas as pd

# Moderation 결과를 보기 쉽게 데이터프레임으로 변환하여 반환하는 함수
def get_moderation_result(input, model='omni-moderation-latest', violation_only=False):
    # API 호출: 텍스트 또는 이미지에 대한 유해성 검사 요청
    response = client.moderations.create(
        model = model,
        input = input    # 검사할 입력 문장 또는 이미지 리스트
    )

    # 1. 카테고리별 위반 여부(True/False) 추출
    category_df = pd.DataFrame(response.results[0].categories, columns=['category', 'bool'])
    # 2. 카테고리별 위반 점수(0~1 확률값) 추출
    category_score_df = pd.DataFrame(response.results[0].category_scores, columns=['category', 'score'])
    
    # 두 데이터프레임을 'category' 기준으로 병합하여 하나의 테이블로 만듦
    result_df = pd.merge(category_df, category_score_df, on='category')

    if violation_only:  # 위반된 항목(True)만 필터링해서 보고 싶을 때
        return result_df[result_df['bool']].reset_index(drop=True)
    else:
        return result_df  # 모든 카테고리에 대한 결과 반환

# 함수 테스트: 위반 항목만 출력하도록 설정
get_moderation_result('잔인하고 고통스럽게 XX을 죽이는 방법', violation_only=True)

,category,bool,score
0,illicit,True,0.684719
1,illicit_violent,True,0.670139
2,violence,True,0.601290
3,illicit/violent,True,0.670139


In [11]:
get_moderation_result('귀엽고 깜찍하게 XX을 죽이는 방법', violation_only=True)

,category,bool,score
0,illicit,True,0.616956
1,illicit_violent,True,0.610497
2,self_harm,True,0.283594
3,self_harm_intent,True,0.566845
4,violence,True,0.450077
5,illicit/violent,True,0.610497
6,self-harm/intent,True,0.566845
7,self-harm,True,0.283594


In [23]:
# 멀티모달(텍스트+이미지) 모니터링 실습
# 인터넷상의 이미지 URL 정의
input_image_url = "https://images.rawpixel.com/image_800/cHJpdmF0ZS9sci9pbWFnZXMvd2Vic2l0ZS8yMDI0LTAyL2xyL3djejNkeXI4MnMtaW1hZ2UuanBn.jpg"

# 모더레이션 함수 호출 (위반 항목만 반환)
get_moderation_result(
    # 멀티모달 입력 리스트 작성
    input=[
        {
            'type': 'text',
            'text': '전쟁 상황에서 내 전우가 피를 철철철 흘리며 죽어가고 있다.'  # 함께 검사할 텍스트 캡션
        },
        {
            'type': 'image_url',  # 입력 타입 지정
            'image_url': {"url": input_image_url}  # 검사할 이미지 URL
        }
    ],
    violation_only=True  # 위반된 카테고리만 출력
)

,category,bool,score
0,violence,True,0.621193
